This Jupyter Notebook is a follow-up to the predictive model notebook ("Education_data_analysis.ipynb") and aims to evaluate the fairness of the trained model on the fictional educational dataset. The predictive model was designed to identify which variables are significant in predicting whether a student would be promoted or retained. The dataset includes 657 observations across 14 columns/variables and includes both continuous and categorical variables such as Gender, Race, Grade, Attendance Rate, Free or Reduced Lunch, Primary Language, Lives with Parent/Guardian, Promoted, SEL Score, Reading Level, Case Managed, and Average Grade.

In the previous notebook, the data was loaded, cleaned, and split into training and testing sets before creating a decision tree model. The model identified that being case managed, having a low attendance rate, being at-risk of dropping out, and identifying as male were significant variables in predicting promotion to the next grade; however, it's unclear whether the model is fair and unbiased for different subgroups of students, such as those of different genders or races.

To address this issue, this notebook will evaluate the fairness of the trained model by loading the preprocessed data and trained model from the previous notebook. First, the evaluation metrics will be defined, depending on the fairness metrics you want to use. For example, you might want to calculate the false positive rates or false negative rates for different groups or calculate the overall accuracy of the model on different groups.

Next, the model's performance will be evaluated using the defined evaluation metrics on different groups to identify any disparities or biases in the model's predictions. Depending on the disparities or biases identified, potential solutions will be explored, such as re-sampling the data to balance the representation of different groups or adjusting the model's hyperparameters.

Finally, the revised model will be tested to see if it performs better on different groups using the same or similar evaluation metrics to compare the performance of the revised model with the original model. By doing so, this notebook aims to identify any disparities or biases in the original model and explore potential solutions to improve its fairness and accuracy.

In [1]:
import pickle
import pandas as pd

# Load the preprocessed data
with open('preprocessed_data.pkl', 'rb') as f:
    edu = pickle.load(f)

# Load the trained model
with open('trained_model.pkl', 'rb') as f:
    edu_dec_tree = pickle.load(f)
    

In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Define the sensitive attributes
sensitive_attrs = ['Gender_Female', 'Gender_Male', 'Race_African American', 'Race_Asian', 
                   'Race_Hispanic', 'Race_White', 'Primary_lang_English', 'Primary_lang_Spanish']

# Create the sensitive feature data
X_sensitive = edu[sensitive_attrs]

# Create the labels for the sensitive feature data
y_sensitive = edu['Promoted']

# Create the group membership for each data point
group_membership = X_sensitive.apply(lambda x: tuple(x), axis=1)

# Get the indices for each group
group_indices = {group: data.index.tolist() for group, data in edu.groupby(group_membership)}



In [6]:
# Define the evaluation metrics
def evaluate_fairness(y_true, y_pred, group_membership):
    confusion_matrices = {}
    accuracies = {}
    for group, indices in group_indices.items():
        # Convert y_pred to a pandas Series
        y_pred_group = pd.Series(y_pred[indices])
        confusion_matrices[group] = confusion_matrix(y_true.iloc[indices], y_pred_group)
        accuracies[group] = accuracy_score(y_true.iloc[indices], y_pred_group)
    return confusion_matrices, accuracies

# Evaluate the fairness of the model on the test set
X_test = edu.drop('Promoted', axis=1)
y_test = edu['Promoted']
y_pred = edu_dec_tree.predict(X_test)
cm, acc = evaluate_fairness(y_test, y_pred, group_membership)

# Print overall accuracy and confusion matrix
print(f"Overall accuracy: {accuracy_score(y_test, y_pred)}")
print("Confusion matrix:")
print(cm)


Overall accuracy: 0.9360730593607306
Confusion matrix:
{(0, 1, 0, 0, 0, 0, 1, 0): array([[14,  1],
       [ 0, 41]]), (0, 1, 0, 0, 0, 1, 0, 1): array([[1]]), (0, 1, 0, 0, 0, 1, 1, 0): array([[ 10,   4],
       [  0, 145]]), (0, 1, 0, 0, 1, 0, 0, 1): array([[ 9,  1],
       [ 2, 38]]), (0, 1, 0, 0, 1, 0, 1, 0): array([[ 2,  0],
       [ 0, 13]]), (0, 1, 0, 1, 0, 0, 1, 0): array([[ 0,  5],
       [ 0, 18]]), (0, 1, 1, 0, 0, 0, 1, 0): array([[ 1,  9],
       [ 1, 14]]), (1, 0, 0, 0, 0, 0, 1, 0): array([[ 6,  1],
       [ 0, 31]]), (1, 0, 0, 0, 0, 1, 0, 1): array([[3]]), (1, 0, 0, 0, 0, 1, 1, 0): array([[  7,   8],
       [  2, 144]]), (1, 0, 0, 0, 1, 0, 0, 1): array([[12,  4],
       [ 0, 46]]), (1, 0, 0, 1, 0, 0, 1, 0): array([[ 1,  3],
       [ 0, 50]]), (1, 0, 1, 0, 0, 0, 1, 0): array([[1, 1],
       [0, 8]])}


This output shows the overall accuracy of the model and the confusion matrices for each group of individuals defined by the group_membership variable.

The overall accuracy of the model is 0.936, which means that the model predicted the correct promotion outcome for 93.6% of the individuals in the test set.

The confusion matrices show the number of true positives, false positives, true negatives, and false negatives for each group of individuals. For example, the first group represented by the tuple (0, 1, 0, 0, 0, 0, 1, 0) has 14 true negatives and 41 true positives. 

This means that the model correctly predicted that 14 individuals in this group were not promoted (true negatives) and incorrectly predicted that one individual was promoted when they were not (false positive). The model correctly predicted that 41 individuals in this group were promoted (true positives). There were no false negatives in this group, which means that the model correctly predicted that all individuals who were promoted were in fact promoted.

The confusion matrices can be utilized to calculate a number of fairness metrics (i.e., false positive rate, false negative rate, equalized odds, impact ratio, etc.). For this project, the focus will be on equalized odds (a measure of whether similar true positive rates and false positive rates hold true across groups) and impact ratio (a measure of the proportion of positive outcomes in each group to then look at any significant differences in outcomes for the groups).

In [9]:
from fairlearn.metrics import equalized_odds_difference, selection_rate_ratio

# Calculate equalized odds difference
eod = equalized_odds_difference(y_test, y_pred, sensitive_features=group_membership)
print("Equalized Odds Difference:", eod)

# Calculate selection rate ratio
srr = selection_rate_ratio(y_test, y_pred, sensitive_features=group_membership)
print("Selection Rate Ratio:", srr)


Equalized Odds Difference: 1.0
Selection Rate Ratio: 0.75


The Equalized Odds Difference of 1.0 means there is a significant difference in the error rates for the different groups. This is indication that the model is not fair with respect to the sensitive attributes.

The Selection Rate Ratio of 0.75 indicates that the selection rate (percentage of individuals in the protected group who were select for promotion) is 0.75 times that of the non-protected group. This suggests that there is a potential disparate impact in the promotion process.

Further analysis is warranted based on these metrics. Some examples would be:
Intersectional Analysis: This involves examining the impact of the intersection of different protected characteristics on the outcomes. For example, how does gender and race interact to impact promotions in the organization? This analysis would involve creating subgroups based on the intersection of different protected characteristics and examining the outcomes for each subgroup.

Counterfactual Analysis: This involves simulating different scenarios to identify what would happen if certain factors were changed. For example, what would happen if the hiring committee used blind hiring techniques to avoid being influenced by the gender or race of the candidates? This analysis would involve simulating different scenarios and comparing the outcomes to identify the impact of different factors.

Causal Inference: This involves identifying causal relationships between different factors and the outcomes. For example, does having a higher education level cause an employee to be more likely to get a promotion? This analysis would involve identifying causal relationships through statistical methods such as regression analysis or experimental design.